In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import altair as alt
from lib.utils import test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Datasets

In [8]:
print(path_data / "train.csv")

data\train.csv


In [9]:
import os

In [13]:
from pathlib import Path
path_data = Path("../data")
train = pd.read_csv(path_data / "train.csv")
test = pd.read_csv(path_data / "test.csv")

print(train.shape, test.shape)

(140700, 20) (93800, 19)


In [19]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [18]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  object 
 2   Gender                                 93800 non-null  object 
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  object 
 5   Working Professional or Student        93800 non-null  object 
 6   Profession                             69168 non-null  object 
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

In [20]:
train.head(3)

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1


In [21]:
test.head(3)

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No


### nan-values relative to dataset size.
- probably because of logical reasoning, e.g., a full-time student without a job has 0 work pressure or a full time worker has 0 pressure in studies. basically if there are no real values missing there should be 20% academics and 80 workers (expect for people where both is NaN, but if both is NaN it could also point to pressure from other life areas...)

In [ ]:
train.isna().sum().div(train.shape[0]).mul(100).round(2)

id                                        0.00
Name                                      0.00
Gender                                    0.00
Age                                       0.00
City                                      0.00
Working Professional or Student           0.00
Profession                               26.03
Academic Pressure                        80.17
Work Pressure                            19.84
CGPA                                     80.17
Study Satisfaction                       80.17
Job Satisfaction                         19.84
Sleep Duration                            0.00
Dietary Habits                            0.00
Degree                                    0.00
Have you ever had suicidal thoughts ?     0.00
Work/Study Hours                          0.00
Financial Stress                          0.00
Family History of Mental Illness          0.00
Depression                                0.00
dtype: float64

In [26]:
test.isna().sum().div(train.shape[0]).mul(100).round(2)

id                                        0.00
Name                                      0.00
Gender                                    0.00
Age                                       0.00
City                                      0.00
Working Professional or Student           0.00
Profession                               17.51
Academic Pressure                        53.33
Work Pressure                            13.35
CGPA                                     53.33
Study Satisfaction                       53.33
Job Satisfaction                         13.34
Sleep Duration                            0.00
Dietary Habits                            0.00
Degree                                    0.00
Have you ever had suicidal thoughts ?     0.00
Work/Study Hours                          0.00
Financial Stress                          0.00
Family History of Mental Illness          0.00
dtype: float64

In [28]:
test.select_dtypes(exclude="object").describe()

,id,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress
count,93800.000000,93800.000000,18767.000000,75022.000000,18766.000000,18767.000000,75026.00000,93800.000000,93800.000000
mean,187599.500000,40.321685,3.158576,3.011797,7.674016,2.939522,2.96092,6.247335,2.978763
std,27077.871962,12.393480,1.386666,1.403563,1.465056,1.374242,1.41071,3.858191,1.414604
min,140700.000000,18.000000,1.000000,1.000000,5.030000,1.000000,1.00000,0.000000,1.000000
25%,164149.750000,29.000000,2.000000,2.000000,6.330000,2.000000,2.00000,3.000000,2.000000
50%,187599.500000,42.000000,3.000000,3.000000,7.800000,3.000000,3.00000,6.000000,3.000000
75%,211049.250000,51.000000,4.000000,4.000000,8.940000,4.000000,4.00000,10.000000,4.000000
max,234499.000000,60.000000,5.000000,5.000000,10.000000,5.000000,5.00000,12.000000,5.000000


In [29]:
train.select_dtypes(exclude="object").describe()

,id,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,140700.000000,140700.000000,27897.000000,112782.000000,27898.000000,27897.000000,112790.000000,140700.000000,140696.000000,140700.000000
mean,70349.500000,40.388621,3.142273,2.998998,7.658636,2.944940,2.974404,6.252679,2.988983,0.181713
std,40616.735775,12.384099,1.380457,1.405771,1.464466,1.360197,1.416078,3.853615,1.413633,0.385609
min,0.000000,18.000000,1.000000,1.000000,5.030000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,35174.750000,29.000000,2.000000,2.000000,6.290000,2.000000,2.000000,3.000000,2.000000,0.000000
50%,70349.500000,42.000000,3.000000,3.000000,7.770000,3.000000,3.000000,6.000000,3.000000,0.000000
75%,105524.250000,51.000000,4.000000,4.000000,8.920000,4.000000,4.000000,10.000000,4.000000,0.000000
max,140699.000000,60.000000,5.000000,5.000000,10.000000,5.000000,5.000000,12.000000,5.000000,1.000000


### Observations
1. age: 18 to 60
2. Depression is an not the status quo -> 20%
3. Average Person is content with their job

## Check Object Column Types
- Analyze Value Ranges: How many unique values? How are the distribution?

### Observations
1. Cities Column: High difference in category 98 Values -> Maybe add additional information like country or continent.
2. Profession Type: 2 Values, Student or Worker -> OK
3. Professions: 64 values: A lot of Abbreviations
4. Sleep Duration: 36 Different Types, could be compressed.
5. Dietary Habits: 23 Different Habits + dirty values.
6. Dirty Values, 115 Degrees, probably could be aggregated.
7. Suicidal Thought: 2 Categorical Values -> OK
8. Family History of Mental Illness: 2 Categorical Values -> OK

In [37]:
for col_name in train.select_dtypes(include="object").columns:
    # skip columns where value ranges don't matter.
    if col_name in ["Name"]:
        continue
    print(col_name, train[col_name].nunique(),  train[col_name].unique())
    

Gender 2 ['Female' 'Male']
City 98 ['Ludhiana' 'Varanasi' 'Visakhapatnam' 'Mumbai' 'Kanpur' 'Ahmedabad'
 'Thane' 'Nashik' 'Bangalore' 'Patna' 'Rajkot' 'Jaipur' 'Pune' 'Lucknow'
 'Meerut' 'Agra' 'Surat' 'Faridabad' 'Hyderabad' 'Srinagar' 'Ghaziabad'
 'Kolkata' 'Chennai' 'Kalyan' 'Nagpur' 'Vadodara' 'Vasai-Virar' 'Delhi'
 'Bhopal' 'Indore' 'Ishanabad' 'Vidhi' 'Ayush' 'Gurgaon' 'Krishna'
 'Aishwarya' 'Keshav' 'Harsha' 'Nalini' 'Aditya' 'Malyansh' 'Raghavendra'
 'Saanvi' 'M.Tech' 'Bhavna' 'Less Delhi' 'Nandini' 'M.Com' 'Plata'
 'Atharv' 'Pratyush' 'City' '3.0' 'Less than 5 Kalyan' 'MCA' 'Mira'
 'Moreadhyay' 'Morena' 'Ishkarsh' 'Kashk' 'Mihir' 'Vidya' 'Tolkata' 'Anvi'
 'Krinda' 'Ayansh' 'Shrey' 'Ivaan' 'Vaanya' 'Gaurav' 'Harsh' 'Reyansh'
 'Kashish' 'Kibara' 'Vaishnavi' 'Chhavi' 'Parth' 'Mahi' 'Tushar' 'MSc'
 'No' 'Rashi' 'ME' 'Molkata' 'Researcher' 'Kagan' 'Armaan' 'Ithal'
 'Nalyan' 'Dhruv' 'Galesabad' 'Itheg' 'Aaradhya' 'Pooja' 'Khushi'
 'Khaziabad' 'Jhanvi' 'Unirar']
Working Professional 

In [38]:
for col_name in test.select_dtypes(include="object").columns:
    # skip columns where value ranges don't matter.
    if col_name in ["Name"]:
        continue
    print(col_name, train[col_name].nunique(),  train[col_name].unique())
    

Gender 2 ['Female' 'Male']
City 98 ['Ludhiana' 'Varanasi' 'Visakhapatnam' 'Mumbai' 'Kanpur' 'Ahmedabad'
 'Thane' 'Nashik' 'Bangalore' 'Patna' 'Rajkot' 'Jaipur' 'Pune' 'Lucknow'
 'Meerut' 'Agra' 'Surat' 'Faridabad' 'Hyderabad' 'Srinagar' 'Ghaziabad'
 'Kolkata' 'Chennai' 'Kalyan' 'Nagpur' 'Vadodara' 'Vasai-Virar' 'Delhi'
 'Bhopal' 'Indore' 'Ishanabad' 'Vidhi' 'Ayush' 'Gurgaon' 'Krishna'
 'Aishwarya' 'Keshav' 'Harsha' 'Nalini' 'Aditya' 'Malyansh' 'Raghavendra'
 'Saanvi' 'M.Tech' 'Bhavna' 'Less Delhi' 'Nandini' 'M.Com' 'Plata'
 'Atharv' 'Pratyush' 'City' '3.0' 'Less than 5 Kalyan' 'MCA' 'Mira'
 'Moreadhyay' 'Morena' 'Ishkarsh' 'Kashk' 'Mihir' 'Vidya' 'Tolkata' 'Anvi'
 'Krinda' 'Ayansh' 'Shrey' 'Ivaan' 'Vaanya' 'Gaurav' 'Harsh' 'Reyansh'
 'Kashish' 'Kibara' 'Vaishnavi' 'Chhavi' 'Parth' 'Mahi' 'Tushar' 'MSc'
 'No' 'Rashi' 'ME' 'Molkata' 'Researcher' 'Kagan' 'Armaan' 'Ithal'
 'Nalyan' 'Dhruv' 'Galesabad' 'Itheg' 'Aaradhya' 'Pooja' 'Khushi'
 'Khaziabad' 'Jhanvi' 'Unirar']
Working Professional 

### Observation
- Testset has the same amount of unique values for each column with dtype object

In [86]:
from pprint import pformat

for col_name in test.select_dtypes(include="object").columns:
    # skip columns where value ranges don't matter.
    if col_name in ["Name"]:
        continue
    
    differences = set(test[col_name].unique()).symmetric_difference(train[col_name].unique())
    common_vals = len(set(test[col_name].unique()).intersection(train[col_name].unique()))
    differences_str = " ".join(differences)
    print(f"{col_name.center(20, '-')}")
    print(f"N-Different Values:{len(differences):^5} Equal Values: {common_vals:^5}")
    print(f"{pformat(differences_str, width=100)}\n")
    
    

-------Gender-------
N-Different Values:  0   Equal Values:   2  
''

--------City--------
N-Different Values: 82   Equal Values:  42  
('Aaradhya ME Aishwarya Gaurav Nalyan Ishanabad Shrey Sara Khushi Chemist Mhopal Unaly Ishkarsh '
 'Hrithik Kashish Malyansh Ithal San Vasai-Virar Saurav Dhruv 3.0 Siddhesh Vaanya Lawyer Malyan '
 'Gurgaon Khaziabad Krinda Tolkata Abhinav No.12 Thani Ayush Kibara Itheg Aditi Less than 5 Kalyan '
 'Kagan Moreadhyay Anvi Morena Ivaan Kashk Mira Is Kanpur Ayansh Galesabad Reyansh Rolkata Plata '
 'Harsh Golkata More Delhi Rashi Harsha Researcher Chhavi Less than 5 hours Mihir Atharv M.Tech '
 'Armaan M.Com Ghopal Molkata Mahi Unirar Raghavendra Nandini Vikram Tushar MCA MSc Krishna Ira '
 'Vaikot Avni Leela Aditya Pratham Pooja Jhanvi')

Working Professional or Student
N-Different Values:  0   Equal Values:   2  
''

-----Profession-----
N-Different Values: 34   Equal Values:  48  
('B.Pharm ME City Manager MBBS BCA B.Com No Surgeon FamilyVirar 3M Pranav 